## Visuals

In [22]:
import altair as alt
import altair_saver
import intake
import numpy as np
import pandas as pd
import shared_utils
from calitp import *
from shared_utils import altair_utils, styleguide
from siuba import *
pd.set_option('display.max_rows', 500)

In [2]:
parquet = pd.read_parquet(
    "gs://calitp-analytics-data/data-analyses/grants/BlackCat_TIRCP_DLA.parquet",
    engine="auto",
)

In [3]:
parquet.head()

,locode,name,BC_TIRCP_DLA_merge
0,6302.0,Humboldt Bay Harbor Recreation & Conservation ...,DLA Only
1,6330.0,Willow Creek Community Services District,DLA Only
2,5036.0,Trinidad,DLA Only
3,5049.0,Ukiah,DLA Only
4,5082.0,Willits,DLA Only


In [4]:
parquet.name.nunique()

1217

In [20]:
grouped = parquet.groupby(['BC_TIRCP_DLA_merge', 'name']).agg({'locode':'max'})

In [23]:
grouped.head(500)

locode
BC_TIRCP_DLA_merge name                                                      
BlackCat Only      ARC Bakersfield                                        NaN
                   ARC Imperial Valley                                    NaN
                   Able Industries                                        NaN
                   Alegria Community Living                               NaN
                   Alpine County Community Development                    NaN
                   Amador Transit                                         NaN
                   Angel View, Inc.                                       NaN
                   Area 1 Agency on Aging                                 NaN
                   Asian Community Center of Sacramento Valley, In...     NaN
                   Avenidas                                               NaN
                   Bay Area Outreach & Recreation Program, Inc            NaN
                   Big Valley 50 Plus                                     NaN
                   Butte County Association of Governments/ Butte ...     NaN
                   Calaveras County Public Works                          NaN
                   Calaveras Transit Agency                               NaN
                   Care-A-Van Transit Systems, Inc.                       NaN
                   Casa Allegra Community Services                        NaN
                   Catholic Charities of the Diocese of Stockton          NaN
                   Center for Elders' Independence                        NaN
                   Choice in Aging                                        NaN
                   City of Arcata                                         NaN
                   City of Arvin                                          NaN
                   City of Auburn                                         NaN
                   City of California City                                NaN
                   City of Chowchilla                                     NaN
                   City of Corcoran - Corcoran Area Transit               NaN
                   City of Dinuba                                         NaN
                   City of Dixon                                          NaN
                   City of Escalon                                        NaN
                   City of Fairfield                                      NaN
                   City of Fontana                                        NaN
                   City of Guadalupe                                      NaN
                   City of Lafayette: Lamorinda Sprit Van Program         NaN
                   City of McFarland                                      NaN
                   City of Needles                                        NaN
                   City of Ojai                                           NaN
                   City of Petaluma                                       NaN
                   City of Porterville                                    NaN
                   City of Ridgecrest                                     NaN
                   City of Rio Vista                                      NaN
                   City of Roseville                                      NaN
                   City of Santa Maria                                    NaN
                   City of Santa Rosa                                     NaN
                   City of Shafter                                        NaN
                   City of Solvang                                        NaN
                   City of Taft                                           NaN
                   City of Tehachapi                                      NaN
                   City of Visalia                                        NaN
                   City of Vista                                          NaN
                   City of Wasco                                          NaN
               

In [5]:
summary = (
    parquet.BC_TIRCP_DLA_merge.value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "Grants", "BC_TIRCP_DLA_merge": "Count_of_Organizations"})
)

In [6]:
summary

,Grants,Count_of_Organizations
0,DLA Only,991
1,BlackCat Only,168
2,BlackCat and DLA,33
3,TIRCP_Only,21
4,"TIRCP, BlackCat and DLA",17
5,TIRCP and BlackCat,1


In [7]:
legend = [
    "DLA Only",
    "BlackCat Only",
    "BlackCat and DLA",
    "TIRCP Only",
    "TIRCP, BlackCat, and DLA",
    "TIRCP and BlackCat",
]

In [16]:
# Advanced bar chart: label the bars & re order the legend.
def labeling(word):
    # Add specific use cases where it's not just first letter capitalized
    LABEL_DICT = {
        "prepared_y": "Year",
        "dist": "District",
        "nunique": "Number of Unique",
        "project_no": "Project Number",
    }

    if (word == "mpo") or (word == "rtpa"):
        word = word.upper()
    elif word in LABEL_DICT.keys():
        word = LABEL_DICT[word]
    else:
        # word = word.replace('n_', 'Number of ').title()
        word = word.replace("unique_", "Number of Unique ").title()
        word = word.replace("_", " ").title()

    return word


def base_bar(df):
    chart = alt.Chart(df).mark_bar()
    return chart


# Function
def make_bar(
    df, y_col, x_col, label_col, chart_title=""
):  # label_col is the column I want the bar graph to be labeled by.

    if chart_title == "":
        chart_title = (
            f"{labeling(x_col)} by {labeling(y_col)}"  # Allows me to customize title.
        )

    bar = base_bar(df)

    bar = bar.encode(
        x=alt.X(x_col, title=labeling(x_col)),
        y=alt.Y(
            y_col, title=labeling(y_col), sort=("-x")
        ),  # Sorts chart with X column from descending.
        color=alt.Color(
            y_col,
            scale=alt.Scale(
                domain=legend,  # List that specifies the order of the legend.
                range=altair_utils.CALITP_CATEGORY_BRIGHT_COLORS,
            ),
        ),
    )
    # https://stackoverflow.com/questions/54015250/altair-setting-constant-label-color-for-bar-chart
    text = bar.mark_text(align="left", baseline="middle", color="black", dy=3).encode(
        text=label_col,
        # Set color here, because encoding for mark_text gets
        # superseded by alt.Color
        color=alt.value("black"),
    )

    chart = bar + text

    chart = (
        styleguide.preset_chart_config(chart)
        .properties(title=chart_title)
        .configure_axis(grid=False)
    )
    chart.save(f"./bar_{x_col}_by_{y_col}.png")
    display(chart)

![Graph](bar_Count_of_Organizations_by_Grants.png)

In [17]:
make_bar(
    summary,
    "Grants",
    "Count_of_Organizations",
    "Count_of_Organizations",
    "# of Organizations by Grant",
)

E0419 15:47:11.843502878    1177 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0419 15:47:11.862356780    1177 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0419 15:47:11.877823210    1177 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0419 15:47:12.265772206    1177 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


alt.LayerChart(...)